In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import nltk
from nltk.stem import WordNetLemmatizer 

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', shuffle = True)
newsgroups_test = fetch_20newsgroups(subset='test', shuffle = True)

In [3]:
print(list(newsgroups_train.target_names))


['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [5]:
(newsgroups_train.data[:2])


["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [4]:
def preprocess(text):
    return [w for w in gensim.utils.simple_preprocess(text) if w not in gensim.parsing.preprocessing.STOPWORDS and len(w)>3]
def lemmatize(text):
    return [WordNetLemmatizer().lemmatize(w) for w in text]

In [6]:
preproc_doc = []
for s in newsgroups_train.data[:4]:
    preproc_doc.append(lemmatize(preprocess(s)))
    

In [7]:
len(preproc_doc[0])

44

In [8]:
dwords = gensim.corpora.Dictionary(preproc_doc)

In [11]:
c = 0
for k,v in dwords.iteritems():
    c+=1
    print(k,v)
    if c==10:
        break

0 addition
1 body
2 bricklin
3 brought
4 bumper
5 called
6 college
7 door
8 early
9 engine


In [36]:
dwords.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


In [16]:
dwords.id2token[13]

'host'

In [12]:
bow = [dwords.doc2bow(s) for s in preproc_doc]

In [13]:
bow

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 2),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1)],
 [(13, 1),
  (18, 1),
  (25, 1),
  (26, 1),
  (28, 1),
  (35, 1),
  (37, 1),
  (39, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 2),
  (52, 5),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 2),
  (58, 1),
  (59, 2),
  (60, 2),
  (61, 1),
  (62, 2),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 3),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1

In [17]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow)
corpus_tfidf = tfidf[bow]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.16607175561867923),
 (1, 0.16607175561867923),
 (2, 0.16607175561867923),
 (3, 0.16607175561867923),
 (4, 0.16607175561867923),
 (5, 0.16607175561867923),
 (6, 0.16607175561867923),
 (7, 0.33214351123735847),
 (8, 0.16607175561867923),
 (9, 0.16607175561867923),
 (10, 0.16607175561867923),
 (11, 0.16607175561867923),
 (12, 0.16607175561867923),
 (13, 0.034463003076411945),
 (14, 0.08303587780933962),
 (15, 0.034463003076411945),
 (16, 0.16607175561867923),
 (17, 0.33214351123735847),
 (19, 0.16607175561867923),
 (20, 0.08303587780933962),
 (21, 0.16607175561867923),
 (22, 0.16607175561867923),
 (23, 0.16607175561867923),
 (24, 0.16607175561867923),
 (25, 0.034463003076411945),
 (27, 0.16607175561867923),
 (28, 0.034463003076411945),
 (29, 0.16607175561867923),
 (30, 0.16607175561867923),
 (31, 0.16607175561867923),
 (32, 0.16607175561867923),
 (33, 0.16607175561867923),
 (34, 0.16607175561867923),
 (36, 0.16607175561867923),
 (37, 0.034463003076411945),
 (38, 0.08303587780933962

In [18]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=20, id2word=dwords, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"plus" + 0.004*"played" + 0.004*"price" + 0.004*"premium" + 0.004*"powerbook" + 0.004*"post" + 0.004*"prove" + 0.004*"opinion" + 0.004*"perform" + 0.004*"purdue"
Topic: 1 Word: 0.004*"plus" + 0.004*"played" + 0.004*"price" + 0.004*"premium" + 0.004*"powerbook" + 0.004*"post" + 0.004*"prove" + 0.004*"opinion" + 0.004*"perform" + 0.004*"purdue"
Topic: 2 Word: 0.031*"clock" + 0.025*"washington" + 0.020*"poll" + 0.014*"speed" + 0.014*"experience" + 0.014*"carson" + 0.014*"guykuo" + 0.014*"floppy" + 0.014*"upgrade" + 0.009*"sink"
Topic: 3 Word: 0.004*"plus" + 0.004*"played" + 0.004*"price" + 0.004*"premium" + 0.004*"powerbook" + 0.004*"post" + 0.004*"prove" + 0.004*"opinion" + 0.004*"perform" + 0.004*"purdue"
Topic: 4 Word: 0.004*"plus" + 0.004*"played" + 0.004*"price" + 0.004*"premium" + 0.004*"powerbook" + 0.004*"post" + 0.004*"prove" + 0.004*"opinion" + 0.004*"perform" + 0.004*"purdue"
Topic: 5 Word: 0.004*"plus" + 0.004*"played" + 0.004*"price" + 0.004*"premium" + 0

In [19]:
bow_test = dwords.doc2bow(lemmatize(preprocess(newsgroups_test.data[0])))

for index, score in sorted(lda_model_tfidf[bow_test], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

Score: 0.9334943294525146	 Topic: 0.015*"door" + 0.015*"lerxst" + 0.014*"purdue" + 0.011*"anybody" + 0.011*"powerbook" + 0.011*"machine" + 0.011*"display" + 0.011*"heard" + 0.011*"question" + 0.008*"model"


In [20]:
print(newsgroups_test.target[0])


7


In [21]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_tfidf, corpus=bow, texts=preproc_doc)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,16.0,0.9758,"door, lerxst, purdue, anybody, powerbook, mach...","[lerxst, thing, subject, nntp, posting, host, ..."
1,1,2.0,0.9855,"clock, washington, poll, speed, experience, ca...","[guykuo, carson, washington, subject, clock, p..."
2,2,16.0,0.9922,"door, lerxst, purdue, anybody, powerbook, mach...","[twillis, purdue, thomas, willis, subject, que..."
3,3,7.0,0.9841,"harris, weitek, division, jgreen, green, abrax...","[jgreen, amber, green, subject, weitek, organi..."


In [24]:
for x in df_dominant_topic[df_dominant_topic['Dominant_Topic']==16]['Document_No']:
    print(newsgroups_train.data[x])


From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





From: twillis@ec.ecn.purdue.edu (Thomas E Willis)
Subject: PB questions...
Organization: Purdue University Engineering Computer Network
Distribution: usa
Lines: 36

well folks, my mac plus finally gave up the ghost this weekend after
starting life as a 512k way back in 1985.  s

In [23]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, bow, dictionary=lda_model_tfidf.id2word)
vis

C:\Users\jinit\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                                x                             y  topics  \
topic                                                                     
16       (0.07033726979353762+0j)    (-0.003890530691758721+0j)       1   
2       (-0.03583687315833381+0j)      (0.05562143147016337+0j)       2   
7       (-0.04444768368160779+0j)     (-0.05101206451055137+0j)       3   
19     (0.0005851345321413966+0j)    (-4.22844863443095e-05+0j)       4   
8      (0.0005851345321413966+0j)    (-4.22844863443095e-05+0j)       5   
1      (0.0005851345321413966+0j)    (-4.22844863443095e-05+0j)       6   
3      (0.0005851345321413966+0j)    (-4.22844863443095e-05+0j)       7   
4      (0.0005851345321413966+0j)    (-4.22844863443095e-05+0j)       8   
5      (0.0005851345321413934+0j)   (-4.228448634430845e-05+0j)       9   
6       (0.000585134532141396+0j)   (-4.228448634430976e-05+0j)      10   
9      (0.0005851345321413954+0j)  (-4.2284486344310294e-05+0j)      

In [ ]:
def pipeline(doc):
    bow_doc = dwords.doc2bow(lemmatize(preprocess(doc)))
    for index, score in sorted(lda_model_tfidf[bow_test], key=lambda tup: -1*tup[1]):
        print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))
    
    df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_tfidf, corpus=bow, texts=preproc_doc)
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']    
    return [i for i in df_dominant_topic['Keywords'] if i in df_dominant_topic['Text']]